# Dir Structure
```
<your-repo>
|_ dataset
    |_ dataset.yaml
|_ yolov5*

```

In [1]:
# check if cuda is used
import torch
import os
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import shutil

torch.cuda.is_available()

C:\Users\aditi\miniconda3\envs\yolo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
print(torch.version.cuda)

11.3


In [ ]:
!python -m torch.utils.collect_env

In [6]:
# create dataset.yaml file
if not path.exists('datasets'):
    os.mkdir('datasets')

In [28]:
from PIL import Image
import matplotlib.pyplot as plt

# preprocess raw images
def load_image(path):
    size = (1280, 720)
    img = Image.open(path).convert('RGB')
    return img.resize(size)

# crop image for standard 1280x720
def crop_face(img):
    size = img.size
    center = img.size[0]*0.5
    w = size[1]*0.2
    
    left = center-w
    right = center+w
    top = 75
    bottom = 500
    
    img = img.crop((left,top,right,bottom))
    
    return img

def preprocess():
    DIR = './datasets/training/raw'
    SAVE = './datasets/training/images'
    for file in os.listdir(DIR):
        img = load_image(path.join(DIR, file))
        img = crop_face(img)
        img.save(f"{SAVE}/{file.split('.')[0]}_body.jpg")

In [2]:
# check image health (missing labels, missing images, etc.)
def check_labels():
    DIR = 'datasets/training_small'
    results = []
    for file in os.listdir(path.join(DIR, 'labels')):
        name = file.split('.')[0] + '.jpg'
        for root, dirs, files in os.walk(path.join(DIR, 'images')):
            if name not in files:
                results.append(name)
                
    if len(results) == 0:
        return 'labels are healthy'
    return results

def check_images():
    DIR = 'datasets/training_small'
    results = []
    for file in os.listdir(path.join(DIR, 'images')):
        name = file.split('.')[0] + '.txt'
        for root, dirs, files in os.walk(path.join(DIR, 'labels')):
            if name not in files:
                results.append(file)
    return results

def move_files(files):
    source = 'datasets/training_small/images'
    dest = 'datasets/training_small'
    for file in files:
        full_path = path.join(source, file)
        shutil.move(full_path, dest)

print('Images without label')
x = check_images()
print(len(x))
x

Images without label
7


['background_1.jpg',
 'background_2.jpg',
 'background_3.jpg',
 'background_4.jpg',
 'background_5.jpg',
 'background_6.jpg',
 'background_7.jpg']

# Template
`python yolov5/train.py --img 640 --batch 2 --epochs 1 --data datasets/training/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml`

In [ ]:
# training the model (remove mosaic)
python yolov5/train.py --img 640 --batch 2 --epochs 2 --data datasets/training/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml --rect

In [3]:
# config 
img_size = 640
batch_size = 2
epochs = 300
data_loc = 'datasets/training_small/ak.yaml'
weights = 'yolov5/yolov5s.pt'
hyperparam = 'hyp.custom.yaml'

In [ ]:
# training on smaller dataset (experimental)

# medium yolov5
python yolov5/train.py --img 640 --batch 2 --epochs 200 --data datasets/training_small/ak.yaml --weights yolov5/yolov5m.pt --hyp hyp.custom.yaml --rect

# small yolov5
python yolov5/train.py --img 640 --batch 2 --epochs 300 --data datasets/training_small/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml --rect

# warmup
python yolov5/train.py --img 640 --batch 2 --epochs 200 --data datasets/temp/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml --rect

# validation
python yolov5/val.py --weights yolov5/runs/train/exp/weights/best.pt --data datasets/temp/ak.yaml

In [ ]:
!python yolov5/train.py \
    --img {img_size} \
    --batch {batch_size} \
    --epochs {epochs} \
    --data {data_loc} \
    --weights {weights} \
    --hyp {hyperparam} \
    --rect

In [35]:
# inference images need to have dimensions of 640x640
def pre_inference():
    DIR = './datasets/inference/raw'
    SAVE = './datasets/inference/images'
    
    left = (0, 0, 640, 640)
    right = (640, 0, 1280, 640)
    crops = {'left': left, 'right': right}
    
    for file in os.listdir(DIR):
        img = load_image(path.join(DIR, file))
        
        for k, v in crops.items():
            proc = img.crop(v)
            proc.save(f"{SAVE}/{file.split('.')[0]}_{k}.jpg")
        
pre_inference()

In [ ]:
import torch
model_path = 'yolov5/runs/train/training_test/weights/test.onnx'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)



In [3]:
model.conf = 0.02
img = 'datasets/inference/images/pull_3_right.jpg'
result = model(img)

result.print()

result.pandas().xyxy[0]

image 1/1: 640x640 1 hibiscus, 4 lavas
Speed: 9.0ms pre-process, 124.7ms inference, 3.0ms NMS per image at shape (1, 3, 640, 640)


,xmin,ymin,xmax,ymax,confidence,class,name
0,381.912292,199.964035,467.395813,295.345886,0.445480,1,lava
1,258.156494,202.558975,350.251648,310.185486,0.237979,0,hibiscus
2,270.278687,88.331223,589.577209,395.320007,0.027036,1,lava
3,403.665009,183.343689,495.277557,310.580383,0.022632,1,lava
4,359.010101,151.389618,459.982208,319.383850,0.021440,1,lava


In [2]:
best_weights = 'yolov5/runs/train/exp/weights/best.pt'
confidence = 0.5
source_image = 'datasets/inference/images'
device = 'cpu'

!python yolov5/detect.py \
            --weights {best_weights} \
            --img 640 \
            --conf {confidence} \
            --source {source_image} \
            --device {device}

detect: weights=['yolov5/runs/train/exp/weights/best.pt'], source=datasets/inference/images, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-376-g48a8531 Python-3.9.12 torch-1.11.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7166551 parameters, 0 gradients, 16.2 GFLOPs
image 1/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\11629306773920_left.jpg: 640x640 1 arene, 1 click, 1 vigna, Done. (0.197s)
image 2/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\11629306773920_right.jpg: 640x640 1 arene, 2 fangs, 1 meteor, 1 midnight, Done. (0.184s)
image 3/29 D:\Repository\tracking-vision-yolo\d

In [3]:
!python yolov5/export.py --weights yolov5/runs/train/boatdawn_v2/weights/best.pt --include onnx

export: data=D:\Repository\tracking-vision-yolo\yolov5\data\coco128.yaml, weights=['yolov5/runs/train/boatdawn_v2/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v6.1-376-g48a8531 Python-3.9.12 torch-1.11.0+cu113 CPU

Fusing layers... 
Model summary: 213 layers, 7166551 parameters, 0 gradients, 16.2 GFLOPs

PyTorch: starting from yolov5\runs\train\boatdawn_v2\weights\best.pt with output shape (1, 25200, 63) (14.0 MB)

ONNX: starting export with onnx 1.11.0...
ONNX: export success, saved as yolov5\runs\train\boatdawn_v2\weights\best.onnx (27.8 MB)

Export complete (2.33s)
Results saved to D:\Repository\tracking-vision-yolo\yolov5\runs\train\boatdawn_v2\weights
Detect:          python detect.py --weights

In [105]:
import json
import os
from os import path
import requests
from urllib.request import urlopen
import time

# fetch from aceship characters table
def fetch_details(name):
    opsCode, opsName, opsRarity = [0,0,0]
    
    start = time.time()
    url = 'https://raw.githubusercontent.com/Aceship/AN-EN-Tags/master/json/gamedata/en_US/gamedata/excel/character_table.json'
    resp = urlopen(url)
    data = json.load(resp)
    
    print('Fetching data for ' + name + '...', end=' ')
    for i, [opsCode, opsData] in enumerate(data.items()):
        if opsData['name'].lower() == name:
            opsName = opsData['name']
            opsRarity = opsData['rarity'] + 1
            
            end = time.time()
            print(end-start)
            return [opsCode, opsName, opsRarity]
    
    print('not found')
    return [opsCode, opsName, opsRarity]

# converts classes.txt to JSON
def create_dict():
    FILE_PATH = "./datasets/temp/classes.txt"
    f = open(FILE_PATH, "r")
    
    classes_dict = {}
    for i, name in enumerate(f.readlines()):
        opsNameSanitized = name.strip()
        opsActual, opsName, opsRarity = fetch_details(opsNameSanitized)
        
        class_obj = {
            "actual": opsActual,
            "name": opsName,
            "rarity": opsRarity
        }
        classes_dict[i] = class_obj
    
    return classes_dict


In [106]:
jsonDumps = json.dumps(create_dict(), indent=4)

Fetching data for aciddrop... 1.5824553966522217
Fetching data for ambriel... 1.5815391540527344
Fetching data for ansel... 1.4179456233978271
Fetching data for arene... 1.3652067184448242
Fetching data for beagle... 1.3597800731658936
Fetching data for beanstalk... 1.4064269065856934
Fetching data for beehunter... 1.3609580993652344
Fetching data for bubble... 1.411874532699585
Fetching data for cardigan... 1.3669795989990234
Fetching data for catapult... 1.3668832778930664
Fetching data for chestnut... not found
Fetching data for click... 1.5326402187347412
Fetching data for cuora... 1.6258373260498047
Fetching data for cutter... 1.607717514038086
Fetching data for deepcolor... 1.3701674938201904
Fetching data for dobermann... 1.428853988647461
Fetching data for earthspirit... 1.3693666458129883
Fetching data for fang... 1.3596243858337402
Fetching data for frostleaf... 1.387293815612793
Fetching data for gitano... 1.3615498542785645
Fetching data for gravel... 1.4163365364074707
Fet

In [93]:
f = open("classes.json", "w")
f.write(jsonDumps)

5856

In [10]:
import json
import requests
from urllib.request import urlopen

def fetch_ops_list(rarity):
    url = 'https://raw.githubusercontent.com/Aceship/AN-EN-Tags/master/json/gamedata/en_US/gamedata/excel/character_table.json'
    resp = urlopen(url)
    data = json.load(resp)
    
    ops = []
    for i, [actual, obj] in enumerate(data.items()):
        if obj['rarity'] == rarity-1:
            ops.append(obj['name'])
    
    return ops
    
fetch_ops_list(5)

['Ptilopsis',
 'Breeze',
 'Zima',
 'Texas',
 'Chiave',
 'Reed',
 'Wild Mane',
 'Elysium',
 'Blacknight',
 'Swire',
 'Whislash',
 'Franka',
 'Flamebringer',
 'Sharp',
 'Indra',
 'Flint',
 'Lappland',
 'Ayerscarpe',
 'Bibeak',
 'Tachanka',
 'Specter',
 'Broca',
 'Astesia',
 'Sideroca',
 'Akafuyu',
 'La Pluma',
 'Tequila',
 'Blue Poison',
 'Platinum',
 'GreyThroat',
 'Stormeye',
 'April',
 'Kroos the Keen Glint',
 'Meteorite',
 'Sesa',
 'Executor',
 'Aosta',
 'Amiya',
 'Absinthe',
 'Tomimi',
 'Skyfire',
 'Pith',
 'Leizi',
 'Beeswax',
 'Leonhardt',
 'Mint',
 'Iris',
 'Lava the Purgatory',
 'Corroserum',
 'Kjera',
 'Mayer',
 'Scene',
 'Silence',
 'Warfarin',
 'Folinic',
 'Touch',
 'Ceylon',
 'Whisperain',
 'Tuye',
 'Mulberry',
 'Honeyberry',
 'Nearl',
 'Hung',
 'Projekt Red',
 'Waai Fu',
 'Kafka',
 'Mr. Nothing',
 'Liskarm',
 'Croissant',
 'Bison',
 'Vulcan',
 'Asbestos',
 'Shalem',
 'Shalem',
 'Blitz',
 'Heavyrain',
 'Ashlock',
 'Aurora',
 'Provence',
 'Firewatch',
 'Andreana',
 'Toddifons

In [62]:
import json
import requests
from bs4 import BeautifulSoup

# Fetch event list
def fetch_events():
    url = 'https://gamepress.gg/arknights/other/event-and-campaign-list'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    result = soup.find(id="sort-table")

    table = result.find_all('td', class_='views-field')
    
    for e in range(0, 10, 2):
        event_start = table[e].find_all('time')[0].text
        event_end = table[e].find_all('time')[1].text if len(table[e].find_all('time')) > 1 else 'inf'
        event_name = table[e+1].find_all('a')[-1].text
        
        print(f"Time: {event_start} - {event_end}")
        print(f"Name: {event_name.split(' - ')[0]}", end='\n\n')
    
    return 0

fetch_events()

Time: 2022-08-18 - 2022-08-25
Name: A Light Spark in Darkness

Time: 2022-07-29 - 2022-08-12
Name: Invitation To Wine

Time: 2022-07-14 - 2022-07-24
Name: [Nine-Colored Deer] Display of Auspices

Time: 2022-07-14 - 2022-07-24
Name: Who Is Real Rerun

Time: 2022-07-14 - inf
Name: Integrated Strategies 2



0